In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
from main import train

faster_rcnn = train()

Epoch 1:  23%|██▎       | 72/313 [00:37<02:02,  1.96it/s, loss=41.3, rpn_cls_loss=0.1, rpn_reg_loss=0.806, rcnn_cls_loss=0.428, rcnn_reg_loss=3.27, accuracy=38.7, offsets=3.73]   

In [ ]:
from torch.nn import functional as F
import torchvision.transforms as transforms
from torch import autograd, nn, optim, utils

from dataset_test import Dataset, color_map
from lib.predictor_efficient import FasterRCNNPredictor

dataset = Dataset(transforms.Compose([transforms.ToTensor()]), min_bboxes=3, max_bboxes=7)
dataloader = utils.data.DataLoader(dataset, batch_size=4, num_workers=12, shuffle=True)

dataloader_iter = iter(dataloader)

In [ ]:
faster_rcnn_predictor = FasterRCNNPredictor(faster_rcnn).cuda()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch

from helper import visualize_anchors, visualize_rpn, visualize_rcnn, img_to_np

img, targets, classes = next(dataloader_iter)
img = img.float().cuda()
targets = targets.detach().float().cuda()
img_id = torch.arange(0, img.size()[0])

nms_reg_rounded, nms_reg, nms_cls, rcnn_reg, rcnn_cls, reg, cls = faster_rcnn_predictor(img, img_id)

i = 1

fig, ax = plt.subplots(figsize=(10, 10))
np_img = img_to_np(img[i])
ax.imshow(np_img)
visualize_rpn(ax, nms_reg.cpu()[i], nms_cls.cpu()[i], img.cpu()[i], [1.0, 1.0, 1.0], draw_all=False)
visualize_rpn(ax, nms_reg_rounded.cpu()[i], nms_cls.cpu()[i], img.cpu()[i], [1.0, 0, 0], draw_all=False)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
np_img = img_to_np(img[i])
ax.imshow(np_img)
visualize_rcnn(ax, reg[i].cpu(), cls[i].cpu(), color_map)
plt.show()